In [5]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.autograd.set_detect_anomaly(True)

# Load housing data
data = pd.read_csv("/home/mw/.cache/kagglehub/datasets/yasserh/housing-prices-dataset/versions/1/Housing.csv")
data = data.dropna()  # Handle missing values

# Identify categorical columns
categorical_columns = data.select_dtypes(include=['object']).columns
print("Categorical columns:", categorical_columns)

# Identify boolean columns




# Encode categorical columns
for col in categorical_columns:
    print(col)
    if data[col].nunique() <= 2:  # Binary columns (e.g., yes/no)
        data[col] = LabelEncoder().fit_transform(data[col])
    else:  # Multiclass columns (e.g., furnishingstatus)
        one_hot = pd.get_dummies(data[col], prefix=col)
        data = pd.concat([data.drop(col, axis=1), one_hot], axis=1)


boolean_columns = data.select_dtypes(include='bool').columns
print("Boolean columns:", boolean_columns)
print(data.select_dtypes(include='bool').columns)
# Convert boolean columns to numeric (0/1)
data[boolean_columns] = data[boolean_columns].astype(int)
# Print data types after encoding
print("Data types after encoding:")
print(data.dtypes)

# Normalize numeric features
numeric_columns = data.select_dtypes(include=['float64']).columns
data[numeric_columns] = (data[numeric_columns] - data[numeric_columns].mean()) / data[numeric_columns].std()

# Define target column
target_column = "price"  # Replace with the actual target column name
features = data.drop(target_column, axis=1)
target = data[target_column]

# Print processed features
print("Processed features:")
print(features.head())

# Convert to PyTorch tensors
features_tensor = torch.tensor(features.values, dtype=torch.float32)
target_tensor = torch.tensor(target.values, dtype=torch.float32).unsqueeze(1)

# Create DataLoader
dataset = TensorDataset(features_tensor, target_tensor)
data_loader = DataLoader(dataset, batch_size=128, shuffle=True)


Categorical columns: Index(['mainroad', 'guestroom', 'basement', 'hotwaterheating',
       'airconditioning', 'prefarea', 'furnishingstatus'],
      dtype='object')
mainroad
guestroom
basement
hotwaterheating
airconditioning
prefarea
furnishingstatus
Boolean columns: Index(['furnishingstatus_furnished', 'furnishingstatus_semi-furnished',
       'furnishingstatus_unfurnished'],
      dtype='object')
Index(['furnishingstatus_furnished', 'furnishingstatus_semi-furnished',
       'furnishingstatus_unfurnished'],
      dtype='object')
Data types after encoding:
price                              int64
area                               int64
bedrooms                           int64
bathrooms                          int64
stories                            int64
mainroad                           int64
guestroom                          int64
basement                           int64
hotwaterheating                    int64
airconditioning                    int64
parking                    

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.autograd.set_detect_anomaly(True)

# Initialize weights
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

# Encoder module
class Encoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.bn1 = nn.BatchNorm1d(256, momentum=0.9)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128, momentum=0.9)
        self.relu = nn.LeakyReLU(0.2, inplace=False)
        self.fc_mean = nn.Linear(128, latent_dim)
        self.fc_logvar = nn.Linear(128, latent_dim)

    def forward(self, x):
        h1 = self.fc1(x)
        h1_bn = self.bn1(h1)
        h1_relu = self.relu(h1_bn)

        h2 = self.fc2(h1_relu)
        h2_bn = self.bn2(h2)
        h2_relu = self.relu(h2_bn)

        mean = self.fc_mean(h2_relu)
        logvar = self.fc_logvar(h2_relu)

        return mean, logvar

# Decoder module
class Decoder(nn.Module):
    def __init__(self, latent_dim, output_dim):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(latent_dim, 128)
        self.bn1 = nn.BatchNorm1d(128, momentum=0.9)
        self.fc2 = nn.Linear(128, 256)
        self.bn2 = nn.BatchNorm1d(256, momentum=0.9)
        self.fc3 = nn.Linear(256, output_dim)
        self.relu = nn.LeakyReLU(0.2, inplace=False)

    def forward(self, x):
        h1 = self.fc1(x)
        h1_bn = self.bn1(h1)
        h1_relu = self.relu(h1_bn)

        h2 = self.fc2(h1_relu)
        h2_bn = self.bn2(h2)
        h2_relu = self.relu(h2_bn)

        h3 = self.fc3(h2_relu)
        output = torch.tanh(h3)
        return output

# Discriminator module
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.bn1 = nn.BatchNorm1d(256, momentum=0.9, track_running_stats=True)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128, momentum=0.9, track_running_stats=True)
        self.fc3 = nn.Linear(128, 1)
        self.relu = nn.LeakyReLU(0.2, inplace=False)

    def forward(self, x):
        h1 = self.fc1(x)
        h1_bn = self.bn1(h1)
        h1_relu = self.relu(h1_bn)

        h2 = self.fc2(h1_relu)
        h2_bn = self.bn2(h2)
        h2_relu = self.relu(h2_bn)

        features = h2_relu
        logits = self.fc3(features)
        output = torch.sigmoid(logits)

        return output, features

# VAE-GAN model
class VAE_GAN(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(VAE_GAN, self).__init__()
        self.encoder = Encoder(input_dim, latent_dim)
        self.decoder = Decoder(latent_dim, input_dim)
        self.discriminator = Discriminator(input_dim)

        # Initialize weights
        self.encoder.apply(weights_init)
        self.decoder.apply(weights_init)
        self.discriminator.apply(weights_init)

    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(0.5 * logvar)
            eps = torch.randn_like(std)
            z = mu + eps * std
            return z
        return mu

    def forward(self, x):
        mu, logvar = self.encoder(x)
        z = self.reparameterize(mu, logvar)
        recon_x = self.decoder(z)
        return mu, logvar, recon_x

# Debugging hooks
    def register_hooks(model):
        def hook_fn(grad):
            print(f"Gradient shape: {grad.shape}, Norm: {grad.norm().item()}")

        for name, param in model.named_parameters():
            if param.requires_grad:
                param.register_hook(hook_fn)


In [7]:
def train_step(vae_gan, real_data, optim_E, optim_D, optim_Dis, criterion, gamma, latent_dim):
    batch_size = real_data.size(0)
    device = real_data.device
    
    # Labels
    real_labels = torch.ones(batch_size, 1).to(device)
    fake_labels = torch.zeros(batch_size, 1).to(device)

    # ======= Train Discriminator =======
    optim_Dis.zero_grad(set_to_none=True)  # More efficient than zero_grad()
    
    # Get real scores
    real_scores, real_features = vae_gan.discriminator(real_data)
    d_real_loss = criterion(real_scores, real_labels)
    
    # Get fake scores
    mean, logvar, fake_data = vae_gan(real_data)
    fake_scores, fake_features = vae_gan.discriminator(fake_data.detach())  # Important: detach here
    d_fake_loss = criterion(fake_scores, fake_labels)
    
    # Total discriminator loss
    d_loss = d_real_loss + d_fake_loss
    d_loss.backward()
    optim_Dis.step()

    # ======= Train Generator =======
    optim_D.zero_grad(set_to_none=True)
    
    # Compute reconstruction loss
    fake_scores_g, fake_features_g = vae_gan.discriminator(fake_data)  # Don't detach here
    rec_loss = F.mse_loss(fake_features_g, real_features.detach())
    
    # Generator loss (fool discriminator)
    g_loss = criterion(fake_scores_g, real_labels)
    
    # Total generator loss
    g_total_loss = gamma * rec_loss + g_loss
    g_total_loss.backward(retain_graph=True)  # Need retain_graph for encoder
    optim_D.step()

    # ======= Train Encoder =======
    optim_E.zero_grad(set_to_none=True)
    
    # KL divergence
    kl_div = -0.5 * torch.sum(1 + logvar - mean.pow(2) - logvar.exp(), dim=1).mean()
    
    # Total encoder loss
    e_loss = kl_div + 5 * rec_loss
    e_loss.backward()
    optim_E.step()

    return {
        'd_loss': d_loss.item(),
        'g_loss': g_loss.item(),
        'kl_loss': kl_div.item(),
        'rec_loss': rec_loss.item()
    }

# Main training loop


In [8]:
# Model setup
input_dim = features.shape[1]
latent_dim = 128
vae_gan = VAE_GAN(input_dim=input_dim, latent_dim=latent_dim).to(device)
epochs = 100
lr = 3e-4
alpha = 0.1
gamma = 15
# Optimizers
criterion = nn.BCELoss()
optim_E = torch.optim.RMSprop(vae_gan.encoder.parameters(), lr=lr)
optim_D = torch.optim.RMSprop(vae_gan.decoder.parameters(), lr=lr)
optim_Dis = torch.optim.RMSprop(vae_gan.discriminator.parameters(), lr=lr * alpha)

# Training loop
for epoch in range(epochs):
    for i, (data, _) in enumerate(data_loader):
        real_data = data.to(device)
        
        losses = train_step(
            vae_gan=vae_gan,
            real_data=real_data,
            optim_E=optim_E,
            optim_D=optim_D,
            optim_Dis=optim_Dis,
            criterion=criterion,
            gamma=gamma,
            latent_dim=latent_dim
        )
        
        if i % 50 == 0:
            print(f"[{epoch}/{epochs}][{i}/{len(data_loader)}] "
                  f"D Loss: {losses['d_loss']:.4f} | "
                  f"G Loss: {losses['g_loss']:.4f} | "
                  f"KL Loss: {losses['kl_loss']:.4f} | "
                  f"Rec Loss: {losses['rec_loss']:.4f}")

/home/mw/miniconda3/envs/sih/lib/python3.12/site-packages/torch/autograd/graph.py:825: UserWarning: Error detected in AddmmBackward0. Traceback of forward call that caused the error:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/mw/miniconda3/envs/sih/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/mw/miniconda3/envs/sih/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/mw/miniconda3/envs/sih/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/home/mw/miniconda3/envs/sih/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/home/mw/miniconda3/envs/sih/lib/python3.12/asyncio/base_events.py", line 641, in run_forever
    self._run_once()
  File "/home/mw

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.cuda.FloatTensor [256, 14]], which is output 0 of AsStridedBackward0, is at version 2; expected version 1 instead. Hint: the backtrace further above shows the operation that failed to compute its gradient. The variable in question was changed in there or anywhere later. Good luck!

In [ ]:

# Inside training loop, after computing losses:


# After training
def plot_losses(losses_dict):
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    for (title, losses), ax in zip(losses_dict.items(), axes.flat):
        ax.plot(losses)
        ax.set_title(title)
        ax.set_xlabel('Iterations')
        ax.set_ylabel('Loss')
    plt.tight_layout()
    plt.show()

plot_losses({
    'Discriminator Loss': d_losses,
    'Generator Loss': g_losses,
    'KL Loss': kl_losses,
    'Reconstruction Loss': rec_losses
})

NameError: name 'd_losses' is not defined